In [ ]:
import os
import math
import torch
#from torch.autograd import Variable
#from torch.optim import Adam
#from torch import nn
#import torch.nn.functional as F
#from torchvision import transforms, models
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
from training_loop import train_model

In [ ]:
CSV_FILE = os.path.join('data', 'sample_labels.csv')

In [ ]:
import csv

class XRayDataset(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.files = []
        if not os.path.exists(CSV_FILE):
            raise Exception('missing csv data file {}, please download data as described in README.md'.format(CSV_FILE))

        self.labels = set()
            
        with open(CSV_FILE) as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            next(reader) # skip header
            for row in reader:
                filename, labels, *_ = row
                labels = labels.split('|')
                self.files.append((filename, labels))
                self.labels.update(labels)
        
        # convert set to list to have a guaranteed iteration order
        # this should also be the case with a set, but it is not explictly defined
        self.labels = list(self.labels)

    def __getitem__(self, index):
        filename, labels = self.files[index]
        image = Image.open(os.path.join('data', 'images', filename))
        if self.transform:
            image = self.transform(image)
        
        label_tensor = []
        for label in self.labels:
            if label in labels:
                label_tensor.append(1.0)
            else:
                label_tensor.append(0.0)
        return image, torch.tensor(label_tensor, dtype=torch.float)